In [2]:
from mmdet.apis import init_detector, inference_detector
device = 'cpu'

In [3]:
!mim download mmdet --config faster_rcnn_r50_fpn_1x_coco --dest ./checkpoints

Traceback (most recent call last):
  File "/opt/conda/bin/mim", line 8, in <module>
    sys.exit(cli())
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1404, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 760, in invoke
    return __callback(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/mim/commands/download.py", line 57, in cli
    download(package, configs, dest_root, check_certificate)
  File "/opt/conda/lib/python3.7/site-packages/mim/commands/download.py", line 101, in download
    highlighted_

In [4]:
!mim download mmdet --config mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco --dest ./checkpoints


Traceback (most recent call last):
  File "/opt/conda/bin/mim", line 8, in <module>
    sys.exit(cli())
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1404, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 760, in invoke
    return __callback(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/mim/commands/download.py", line 57, in cli
    download(package, configs, dest_root, check_certificate)
  File "/opt/conda/lib/python3.7/site-packages/mim/commands/download.py", line 101, in download
    highlighted_

In [4]:
!ls configs

ls: cannot access 'configs': No such file or directory


In [5]:
!ls

Untitled.ipynb	checkpoints


In [6]:
!ls ../

bin   dev  home  lib64	mmdetection  notebook  proc  run   srv	tmp  var
boot  etc  lib	 media	mnt	     opt       root  sbin  sys	usr  workspace


In [7]:
!ls ../mmdetection

CITATION.cff	 configs  mmdet.egg-info   requirements.txt  tools
LICENSE		 demo	  model-index.yml  resources
MANIFEST.in	 docker   projects	   setup.cfg
README.md	 docs	  pytest.ini	   setup.py
README_zh-CN.md  mmdet	  requirements	   tests


In [10]:
!wget -c https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_1x_coco/mask_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.38__segm_mAP-0.344_20200504_231812-0ebd1859.pth \
      -O checkpoints/mask_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.38__segm_mAP-0.344_20200504_231812-0ebd1859.pth

--2023-05-24 09:54:44--  https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_1x_coco/mask_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.38__segm_mAP-0.344_20200504_231812-0ebd1859.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 163.181.39.208, 163.181.39.207, 163.181.39.209, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|163.181.39.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177866833 (170M) [application/octet-stream]
Saving to: ‘checkpoints/mask_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.38__segm_mAP-0.344_20200504_231812-0ebd1859.pth’

checkpoints/mask_rc 100%[===================>] 169.63M  1.56MB/s    in 4m 6s   

2023-05-24 09:58:49 (707 KB/s) - ‘checkpoints/mask_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.38__segm_mAP-0.344_20200504_231812-0ebd1859.pth’ saved [177866833/177866833]



In [14]:
from mmdet.apis import inference_detector, init_detector
config = '/mmdetection/configs/mask_rcnn/mask-rcnn_r50-caffe_fpn_1x_coco.py'
checkpoint = 'checkpoints/mask_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.38__segm_mAP-0.344_20200504_231812-0ebd1859.pth'
model = init_detector(config, checkpoint, device='cuda:0')

Loads checkpoint by local backend from path: checkpoints/mask_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.38__segm_mAP-0.344_20200504_231812-0ebd1859.pth


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [9]:
!ls /mmdetection/configs/mask_rcnn

README.md
mask-rcnn_r101-caffe_fpn_1x_coco.py
mask-rcnn_r101-caffe_fpn_ms-poly-3x_coco.py
mask-rcnn_r101_fpn_1x_coco.py
mask-rcnn_r101_fpn_2x_coco.py
mask-rcnn_r101_fpn_8xb8-amp-lsj-200e_coco.py
mask-rcnn_r101_fpn_ms-poly-3x_coco.py
mask-rcnn_r18_fpn_8xb8-amp-lsj-200e_coco.py
mask-rcnn_r50-caffe-c4_1x_coco.py
mask-rcnn_r50-caffe_fpn_1x_coco.py
mask-rcnn_r50-caffe_fpn_ms-1x_coco.py
mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco.py
mask-rcnn_r50-caffe_fpn_ms-poly-2x_coco.py
mask-rcnn_r50-caffe_fpn_ms-poly-3x_coco.py
mask-rcnn_r50-caffe_fpn_poly-1x_coco_v1.py
mask-rcnn_r50_fpn_1x-wandb_coco.py
mask-rcnn_r50_fpn_1x_coco.py
mask-rcnn_r50_fpn_2x_coco.py
mask-rcnn_r50_fpn_8xb8-amp-lsj-200e_coco.py
mask-rcnn_r50_fpn_amp-1x_coco.py
mask-rcnn_r50_fpn_ms-poly-3x_coco.py
mask-rcnn_r50_fpn_poly-1x_coco.py
mask-rcnn_x101-32x4d_fpn_1x_coco.py
mask-rcnn_x101-32x4d_fpn_2x_coco.py
mask-rcnn_x101-32x4d_fpn_ms-poly-3x_coco.py
mask-rcnn_x101-32x8d_fpn_1x_coco.py
mask-rcnn_x101-32x8d_fpn_ms-poly-1x_coco.py
mask-rc

In [15]:
result = inference_detector(model, '/mmdetection/demo/demo.jpg')


NameError: name 'model' is not defined